In [2]:
import pandas as pd              
import datetime as dt     
import mplfinance as mpf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz



In [68]:
# Cargar el DataFrame desde el archivo CSV
df = pd.read_csv("eurusd-5m.csv", sep=';')[-10000:]
df.columns = ['date', 'hour', 'open', 'high', 'low', 'close', 'volume']


# Formato fechas
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df['hour'] = pd.to_datetime(df['hour'], format='%H:%M:%S').dt.time
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['hour'].astype(str))
df = df.drop(columns=["hour", "date"])

# Definir las zonas horarias de origen y destino
gmt_minus_6 = pytz.timezone('Etc/GMT+2')
gmt_plus_2 = pytz.timezone('Etc/GMT-5')

df['datetime_gmt+2'] = df['datetime'].dt.tz_localize(gmt_minus_6).dt.tz_convert(gmt_plus_2)
df['datetime_gmt+2'] = df['datetime_gmt+2'].dt.tz_localize(None)
df.set_index('datetime_gmt+2', inplace=True)
# df = df.drop(columns=["datetime"])

# Sesion de Londres
df['date'] = df.index.date
df['hour'] = df.index.time
df['london'] = '0'
df.loc[(df['hour'] >= pd.to_datetime('09:00:00').time()) & (df['hour'] <= pd.to_datetime('17:25:00').time()), 'london'] = '1'



# Filtrar los valores menores que '2023-06-27'
filtered_df = df[df['date'] < dt.date(2023, 6, 27)]


# Graficar el DataFrame utilizando mplfinance
# mpf.plot(df, type='candle', style='charles', volume=True)

In [69]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'datetime', 'date', 'hour',
       'london'],
      dtype='object')

# Features

In [70]:
df['volume_ma'] = df['volume'].rolling(window=2000).mean()

# Trigger

In [77]:
df['inicio'] = '0'
df['fin'] = '0'
df['trigger'] = '0'


indices_inicio = df[(df['london'] == '1') & (df['london'].shift(1) == '0')].index
indices_fin = df[(df['london'] == '1') & (df['london'].shift(-1) == '0')].index
if len(indices_fin) > len(indices_inicio):
    indices_fin = indices_fin[1:]


# Iterar sobre los índices de inicio de sesión
for indice_inicio in indices_inicio:
    for inicio, fin in zip(indices_inicio, indices_fin):
        indices_sesion = df[(df.index >= inicio) & (df.index <= fin)].index
        primer_indice = indices_sesion[df.loc[indices_sesion, 'volume'] < df.loc[indices_sesion, 'volume_ma']].min()
        # Encontrar el primer índice de la sesión donde se cumple la condición
        primer_indice = df.loc[indice_inicio:, 'volume'].lt(df.loc[indice_inicio:, 'volume_ma']).idxmax()

        df.loc[inicio, 'inicio'] = '1'
        df.loc[fin, 'fin'] = '1'


        if df.loc[primer_indice, 'london'] == '1': 
            df.loc[primer_indice, 'trigger'] = '1'
        else:
            df.loc[primer_indice, 'trigger'] = '0'

i = 0
def etiqueta_sesion(london, inicio):
    global i
    if london == '1' and inicio =='1':
        i = i+1
    elif london == '0':
        return '0'
    return i
df["sesion"] = df.apply(lambda row : etiqueta_sesion(row['london'], row['inicio']), axis=1)
        


In [78]:
df[df.trigger=='1']

,open,high,low,close,volume,datetime,date,hour,london,volume_ma,inicio,fin,trigger,sesion
datetime_gmt+2,,,,,,,,,,,,,,
2023-05-22 11:50:00,1.08195,1.08201,1.08172,1.08193,640,2023-05-22 04:50:00,2023-05-22,11:50:00,1,715.2715,0,0,1,8
2023-05-23 13:35:00,1.07836,1.07842,1.07799,1.07800,703,2023-05-23 06:35:00,2023-05-23,13:35:00,1,711.9030,0,0,1,9
2023-05-25 11:45:00,1.07335,1.07355,1.07327,1.07328,744,2023-05-25 04:45:00,2023-05-25,11:45:00,1,762.8395,0,0,1,11
2023-05-26 12:35:00,1.07353,1.07374,1.07343,1.07374,752,2023-05-26 05:35:00,2023-05-26,12:35:00,1,768.1510,0,0,1,12
2023-05-29 09:05:00,1.07338,1.07340,1.07318,1.07326,731,2023-05-29 02:05:00,2023-05-29,09:05:00,1,762.8415,0,0,1,13
2023-05-31 11:40:00,1.06753,1.06764,1.06738,1.06746,697,2023-05-31 04:40:00,2023-05-31,11:40:00,1,736.8710,0,0,1,15
2023-06-01 11:25:00,1.06900,1.06910,1.06880,1.06904,719,2023-06-01 04:25:00,2023-06-01,11:25:00,1,745.3910,0,0,1,16
2023-06-02 10:10:00,1.07648,1.07658,1.07621,1.07628,651,2023-06-02 03:10:00,2023-06-02,10:10:00,1,738.3670,0,0,1,17
2023-06-05 10:45:00,1.06928,1.06938,1.06915,1.06934,679,2023-06-05 03:45:00,2023-06-05,10:45:00,1,717.7475,0,0,1,18


# Visual

In [41]:
# Crear una figura con subtramas
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0, row_heights=[0.7, 0.3])

#VELAS
# Agregar las velas a la subtrama superior
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close']),
              row=1, col=1)

#VOLUMEN
# Agregar el volumen a la subtrama inferior
fig.add_trace(go.Bar(x=df.index, y=df['volume']),
              row=2, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['volume_ma'], line=dict(color='blue')),
              row=2, col=1)



#SESIÓN LONDRES
precios_max = df[df['london'] == '1']['high']
precios_min = df[df['london'] == '1']['low']


# INICIO-FIN SESIONES
indices_inicio = df[(df['london'] == '1') & (df['london'].shift(1) == '0')].index
indices_fin = df[(df['london'] == '1') & (df['london'].shift(-1) == '0')].index
if len(indices_fin) > len(indices_inicio):
    indices_fin = indices_fin[1:]

# Calcular el rango de precios para ajustar el tamaño de las líneas verticales
rango_precios = max(precios_max) - min(precios_min)
# Agregar líneas verticales para marcar el inicio y el final de la sesión de Londres
for inicio, fin in zip(indices_inicio, indices_fin):
    # Obtener los precios máximo y mínimo en el período de la sesión de Londres
    precio_max_periodo = max(precios_max.loc[inicio:fin])
    precio_min_periodo = min(precios_min.loc[inicio:fin])

    # Ajustar las coordenadas y0 y y1 de las líneas verticales según el rango de precios
    y0 = precio_min_periodo - 0.1 * rango_precios
    y1 = precio_max_periodo + 0.1 * rango_precios

    #APERTURA
    fig.add_shape(
        type='line',
        x0=inicio,
        y0=y0,
        x1=inicio,
        y1=y1,
        line=dict(color='green', width=1, dash='dash'),
    )
    #CIERRE
    fig.add_shape(
        type='line',
        x0=fin,
        y0=y0,
        x1=fin,
        y1=y1,
        line=dict(color='red', width=1, dash='dash'),
    )
    #MÁXIMO
    fig.add_shape(
        type='line',
        x0=inicio,
        y0=precio_max_periodo,
        x1=fin,
        y1=precio_max_periodo,
        line=dict(color='green', width=1, dash='dash'),
    )
    #MÍNIMO
    fig.add_shape(
        type='line',
        x0=inicio,
        y0=precio_min_periodo,
        x1=fin,
        y1=precio_min_periodo,
        line=dict(color='red', width=1, dash='dash'),
    )
    
    #TRIGGER
    indices_sesion = df[(df.index >= inicio) & (df.index <= fin)].index
    primer_indice = indices_sesion[df.loc[indices_sesion, 'volume'] < df.loc[indices_sesion, 'volume_ma']].min()
    if pd.notnull(primer_indice):
        fig.add_shape(
            type='line',
            x0=primer_indice,
            y0=y0,
            x1=primer_indice,
            y1=y1,
            line=dict(color='blue', width=1, dash='dash'),
        )

    #PRECIO APERTURA
    precio_apertura = df.loc[inicio, 'open']
    y = precio_apertura
    fig.add_shape(
        type='line',
        x0=inicio,
        y0=y,
        x1=fin,
        y1=y,
        line=dict(color='black', width=1, dash='dash'),
    )
   
   

    

    
# Habilitar el modo interactivo y las herramientas de zoom para ambas subtramas
fig.update_layout(dragmode='zoom',
                  xaxis_rangeslider_visible=False,
                  height=800)
fig.show()